### 문제 2

#### Import module (코드 수정 X)

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn


from sklearn.model_selection import train_test_split

#### Download dataset (코드 수정 X)

In [ ]:
# Dataset 다운로드
!wget "https://dongaackr-my.sharepoint.com/:x:/g/personal/sjkim_donga_ac_kr/ET2udlQfsxRAsvnlEtgzfi0B3HAMAmqP_Y2WRsbYrTvYaA?e=t6809f&download=1" -O kc_house_data.csv

# Load dataset file
data = pd.read_csv('kc_house_data.csv')
data

#### Data 전처리 (코드 수정 X)

In [ ]:
# 예측 수행에 불필요한 열 삭제
new_data = data.drop(['id', 'date'], axis=1)

# 값이 존재하지 않는 행 삭제
new_data = new_data.dropna()

# 데이터셋 가우시안 정규화
data_normalized = (new_data - new_data.mean()) / new_data.std()
data_normalized

In [ ]:
# 예측에 사용할 데이터들에 대한 2차원 행렬 변환
x = np.array(data_normalized)[:, 1:]
y = np.array(data_normalized[['price']])

# Train dataset / Test dataset 분할
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1234)

# Train dataset 형상 확인
print(x_train.shape)
print(y_train.shape)

#### MLP model

In [ ]:
class MLP(nn.Module):
  def __init__(self):
    super(MLP, self).__init__()

    # Layer 선언

  def forward(self, x):
    
    # MLP 수행

    return y

#### Training (코드 수정 X)

In [ ]:
batch_size = 100
learning_rate = 0.01
training_epochs = 10
loss_function = nn.MSELoss()
network = MLP()
optimizer = torch.optim.SGD(network.parameters(), lr = learning_rate)

network.train()

for epoch in range(training_epochs):
  avg_cost = 0
  total_batch = len(x_train) // batch_size

  for batch in range(total_batch):
    input = torch.tensor(x_train[batch * batch_size : (batch * batch_size) + batch_size, :], dtype=torch.float32)
    label = torch.tensor(y_train[batch * batch_size : (batch * batch_size) + batch_size, :], dtype=torch.float32)

    pred = network(input)

    loss = loss_function(pred, label)
    optimizer.zero_grad() # gradient 초기화
    loss.backward()
    optimizer.step()

    avg_cost += loss / total_batch

  print('Epoch: %d Loss = %f'%(epoch+1, avg_cost))

print('Learning finished')

#### Prediction (코드 수정 X)

In [ ]:
# Test dataset을 이용한 예측
input = torch.tensor(x_test, dtype=torch.float32)
network.eval()
with torch.no_grad(): # test에서는 기울기 계산 제외
    y_hat = network(input)
y_hat = y_hat.detach().numpy()

In [ ]:
# 정답 데이터와 예측 데이터 간 차이 계산
def MSE (pred, label):
  error = pred - label
  mse = np.mean(error ** 2)
  return mse

print(MSE(y_hat, y_test))